# Introduction

## About User Table
A user table ( "takehome_users" ) with data on 12,000 users who signed up for the product in the last two years. 

This table includes:

* name: the user's name
* object_id: the user's id
* email: email address
* creation_source: how their account was created. This takes on one of 5 values:
    * PERSONAL_PROJECTS: invited to join another user's personal workspace
    * GUEST_INVITE: invited to an organization as a guest(limited permissions)
    * ORG_INVITE: invited to an organization (as a full member)
    * SIGNUP: signed up via the website
    * SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login id)
* creation_time: when they created their account
* last_session_creation_time: unix timestamp of last login
* opted_in_to_mailing_list: whether they have opted into receiving marketing emails
* enabled_for_marketing_drip: whether they are on the regular marketing email drip
* org_id: the organization (group of users) they belong to
* invited_by_user_id: which user invited them to join (if applicable).

## About user_engagement Table

A usage summary table ( "takehome_user_engagement" ) that has a row for each day that a user logged into the product.

   **Aim:-**
    To find the features that predict future Adopted user.

   **Definition of Adopted User:-**
    User who has logged into the product on three separate days in at least one sevenday period.

### Imports 

In [69]:
#Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# importing Datetime to change Last_Login dtype
from datetime import datetime


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

### Data Load User Details

In [2]:
os.getcwd()

'C:\\Users\\Avinash Jha\\Documents\\Python\\Springboard\\GITHUB\\relax_challenge'

In [3]:
user = pd.read_csv(r'C:\Users\Avinash Jha\Documents\Python\Springboard\GITHUB\SpringBoard_CaseStudy\relax_challenge\users.csv', encoding = 'latin-1')
user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


Used Encoding as without it the cell raised an UnicodeDecodeError. I used encoding = 'Latin-1' because it worked for me.

#### Changing Column Names

In [4]:
## Changing the names of the columns.
New_Column_Names = ['UserID', 'Account_Creation_Time', 'User_Name', 'User_Email', 'Account_Creation_Method', 
                    'Last_Login', 'Mailing_List', 'Marketing_Dip', 'Organization_ID', 'Referral_By']

user.columns = New_Column_Names
user.head()

,UserID,Account_Creation_Time,User_Name,User_Email,Account_Creation_Method,Last_Login,Mailing_List,Marketing_Dip,Organization_ID,Referral_By
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


#### Setting UserID as Index

In [5]:
#I added it just for visual purpose. Commented this as later I delete it in my feature selection.
#user.set_index('UserID', inplace = True)

#### Initial EDA

In [6]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UserID                   12000 non-null  int64  
 1   Account_Creation_Time    12000 non-null  object 
 2   User_Name                12000 non-null  object 
 3   User_Email               12000 non-null  object 
 4   Account_Creation_Method  12000 non-null  object 
 5   Last_Login               8823 non-null   float64
 6   Mailing_List             12000 non-null  int64  
 7   Marketing_Dip            12000 non-null  int64  
 8   Organization_ID          12000 non-null  int64  
 9   Referral_By              6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


The dataset has 12000 entries. 
last_session_creation_time and invited_by_user_id  has null values

In [7]:
user.describe()

,UserID,Last_Login,Mailing_List,Marketing_Dip,Organization_ID,Referral_By
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [8]:
# Checking for different account creation method for users who were not referred by anyone
user[user.Referral_By.isnull()]['Account_Creation_Method'].unique()

array(['SIGNUP', 'PERSONAL_PROJECTS', 'SIGNUP_GOOGLE_AUTH'], dtype=object)

**Referral By**

There are three different types of Account creation methods used for the users which not have any referral id
* SIGNUP
* PERSONAL_PROJECT
* SGINUP_GOOGLE_AUTH

We can impute the null values by imputing the top member of the organization in regards to referral

#### Last Login

Last login column had unix date time values which needed to be converted to datetime obj.

I used unit parameter as 's' after referring to stack overflow.

Last_Login can be used to create a column which can be used to see the frequency of logins

In [10]:
user.Last_Login.unique()

array([1.39813881e+09, 1.39623750e+09, 1.36373489e+09, ...,
       1.39860272e+09, 1.33863816e+09, 1.39072663e+09])

In [11]:
user['Last_Login'] = pd.to_datetime(user['Last_Login'], unit = 's')

#### User_Emails

The duplicate entries in emails are the results of same individuals registering for app, in different companies or through different referrals. 

In [12]:
len(user.User_Email.unique())

11980

In [13]:
# Listing all the entries with duplicate emails
vEmails = user['User_Email']
user[user['User_Email'].isin(vEmails[user['User_Email'].duplicated()])].sort_values('User_Email')

,UserID,Account_Creation_Time,User_Name,User_Email,Account_Creation_Method,Last_Login,Mailing_List,Marketing_Dip,Organization_ID,Referral_By
1603,1604,2014-02-03 04:48:08,Lane Alfie,AlfieLane@yahoo.com,ORG_INVITE,2014-02-06 04:48:08,0,0,0,4128.0
3822,3823,2012-11-22 05:51:29,Lane Alfie,AlfieLane@yahoo.com,ORG_INVITE,2012-11-23 05:51:29,0,0,169,9498.0
5280,5281,2012-09-29 11:54:08,Bach Amanda,AmandaABach@gmail.com,SIGNUP_GOOGLE_AUTH,2014-06-04 11:54:08,0,0,146,NaN
9040,9041,2013-01-03 00:53:47,Bach Amanda,AmandaABach@gmail.com,ORG_INVITE,NaT,0,0,377,6208.0
375,376,2012-08-10 22:27:08,Tye Jacob,JacobTye@gmail.com,SIGNUP_GOOGLE_AUTH,2012-08-10 22:27:08,1,0,104,NaN
5081,5082,2013-06-25 19:31:11,Tye Jacob,JacobTye@gmail.com,SIGNUP_GOOGLE_AUTH,2013-06-27 19:31:11,0,0,298,NaN
3605,3606,2013-05-10 08:31:59,Sankt Janina,JaninaSankt@gmail.com,GUEST_INVITE,2013-05-29 08:31:59,0,0,84,8356.0
1169,1170,2014-04-27 16:48:34,Sankt Janina,JaninaSankt@gmail.com,ORG_INVITE,2014-04-27 16:48:34,1,1,285,623.0
10153,10154,2014-05-06 10:57:37,Neudorf Kerstin,KerstinNeudorf@gmail.com,ORG_INVITE,2014-05-16 10:57:37,0,0,359,4563.0
11244,11245,2014-01-17 18:05:15,Neudorf Kerstin,KerstinNeudorf@gmail.com,SIGNUP,2014-01-17 18:05:15,1,1,68,NaN


#### Account_Creation_Time

Duplicates were found in Account Creation time. Inspecting the same below.

In [14]:
len(user.Account_Creation_Time.unique())

11996

In [15]:
## Listing Rows with duplicate Account creation time
vList = user['Account_Creation_Time'] #Creating a mask
vList[user['Account_Creation_Time'].duplicated()] #applying the mask


8069     2014-02-11 17:57:53
11223    2012-09-01 07:22:09
11821    2012-09-14 21:35:03
11861    2013-05-25 04:35:52
Name: Account_Creation_Time, dtype: object

In [16]:
user[user['Account_Creation_Time'] == '2014-02-11 17:57:53']

,UserID,Account_Creation_Time,User_Name,User_Email,Account_Creation_Method,Last_Login,Mailing_List,Marketing_Dip,Organization_ID,Referral_By
1589,1590,2014-02-11 17:57:53,Walker Milton,MiltonLWalker@cuvox.de,ORG_INVITE,2014-02-12 17:57:53,0,0,41,3510.0
8069,8070,2014-02-11 17:57:53,Lebouef David,DavidELebouef@hotmail.com,GUEST_INVITE,2014-05-29 17:57:53,0,0,119,4877.0


In [17]:
user['Account_Creation_Time'] = pd.to_datetime(user['Account_Creation_Time'])

### Data Load Engagement Table

In [20]:
user_engagement = pd.read_csv(r'C:\Users\Avinash Jha\Documents\Python\Springboard\GITHUB\SpringBoard_CaseStudy\relax_challenge\user_engagement.csv')
user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


#### Initial EDA

In [21]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [22]:
New_Column_Names = ['Login_Timestamp', 'UserID', 'Visited']
user_engagement.columns = New_Column_Names

In [23]:
user_engagement['Login_Timestamp'] = pd.to_datetime(user_engagement['Login_Timestamp'])

In [24]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Login_Timestamp  207917 non-null  datetime64[ns]
 1   UserID           207917 non-null  int64         
 2   Visited          207917 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


#### Creation of Adopted User Variable

In [25]:
# Creating adopted user 
#vDict = dict()
#vDict[vUserid] = list of time stamps
#pd.DatetimeIndex(user_engagement['Login_Timestamp']).year
#user_engagement['Login_Timestamp'].dt.year

In [26]:
#Try 1
from tqdm import tqdm
if(1 == 2):
    df = user_engagement.drop('Visited', axis = 1).head(1000)
    df.sort_values('UserID', inplace = True)
    i = 0
    vDict = dict()
    for user in tqdm(df['UserID'].unique()):
        vList = []
        for index, row in df.iterrows():
            if(row['UserID'] == user):
                vList.append(row['Login_Timestamp'])
        if(len(vList) > 3):
            vDict[user] = vList

In [27]:
df = user_engagement.drop('Visited', axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Login_Timestamp  207917 non-null  datetime64[ns]
 1   UserID           207917 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.2 MB


In [28]:
#tRY 2
df['Login_Timestamp'] = df['Login_Timestamp'].dt.floor('h').astype(np.int64)
df_grouped = df.groupby('UserID')['Login_Timestamp'].rolling(window=3)
b = pd.to_timedelta(df_grouped.max() - df_grouped.min()).dt.days
b[b <= 7]


UserID        
2       9         6.0
        10        5.0
        11        7.0
10      24        7.0
        27        5.0
                 ... 
11988   207898    3.0
        207899    3.0
        207900    3.0
        207901    3.0
        207902    6.0
Name: Login_Timestamp, Length: 167199, dtype: float64

In [29]:
c = list(b[b <= 7].index.get_level_values(0).unique())

#### Addition of Adoption User Column


In [30]:
user['Adopted_User'] = 0

In [31]:
user['Adopted_User'] = user['Adopted_User'].mask(user['UserID'].isin(c), 1)

In [32]:
user['Adopted_User'].unique()

array([0, 1], dtype=int64)

#### Feature Creation

* UserID will not be used for feature selection as it is unique.
* Account creation time and last login can be broken down to year, Q1, Q2, Q3, Q4. 
* User_Name and Email will not be used in feature selection. 
* Account creation method, organization id, will be one hot encoded(drop) as this is Nominal data.
* Mailing_List and Marketing_Dip can be used as it is


In [34]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   UserID                   12000 non-null  int64         
 1   Account_Creation_Time    12000 non-null  datetime64[ns]
 2   User_Name                12000 non-null  object        
 3   User_Email               12000 non-null  object        
 4   Account_Creation_Method  12000 non-null  object        
 5   Last_Login               8823 non-null   datetime64[ns]
 6   Mailing_List             12000 non-null  int64         
 7   Marketing_Dip            12000 non-null  int64         
 8   Organization_ID          12000 non-null  int64         
 9   Referral_By              6417 non-null   float64       
 10  Adopted_User             12000 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 1.0+ MB


In [35]:
user.drop(['UserID', 'User_Name', 'User_Email', 'Referral_By', 'Organization_ID'], inplace = True, axis = 1)

In [36]:
user['Account_Creation_Year'] = user['Account_Creation_Time'].dt.year
user['Account_Creation'] = user['Account_Creation_Time'].dt.month
user['Account_Creation_Year'] = user['Account_Creation_Year'].astype('object')

In [37]:
user['Account_Creation'].unique()

array([ 4, 11,  3,  5,  1, 12,  7, 10,  2,  8,  9,  6], dtype=int64)

In [38]:
user['Account_Creation'].replace({1: 'Q1', 2:'Q1',3: 'Q2', 4:'Q2',5: 'Q2', 6:'Q3',7: 'Q3', 8:'Q3',9: 'Q3', 10:'Q4', 11:'Q4', 12:'Q4'}, inplace = True)


In [39]:
user = pd.get_dummies(user, columns = ['Account_Creation','Account_Creation_Method'], drop_first = True)


In [40]:
total_missing = user['Last_Login'].isna().sum()
total_rows = 12000

missing_percentage = (total_missing/total_rows)*100
print(missing_percentage, "% values are missing")

26.474999999999998 % values are missing


Thought to impute values but did not find a good way

In [41]:
user['Accessed_Atleast_Once'] = 1
user.loc[user['Last_Login'].isna(), 'Accessed_Atleast_Once'] = 0 

In [42]:
user.drop(['Last_Login','Account_Creation_Time'], inplace = True, axis = 1)

In [43]:
#user['Referral_By'] = user['Referral_By'].astype('object')
#user['Referral_By'].fillna('No_Referral')

In [44]:
#user['Organization_ID'] = user['Organization_ID'].astype('object')
#X = pd.get_dummies(user, columns = ['Organization_ID'], drop_first = True)

In [48]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 12 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Mailing_List                                12000 non-null  int64 
 1   Marketing_Dip                               12000 non-null  int64 
 2   Adopted_User                                12000 non-null  int64 
 3   Account_Creation_Year                       12000 non-null  object
 4   Account_Creation_Q2                         12000 non-null  uint8 
 5   Account_Creation_Q3                         12000 non-null  uint8 
 6   Account_Creation_Q4                         12000 non-null  uint8 
 7   Account_Creation_Method_ORG_INVITE          12000 non-null  uint8 
 8   Account_Creation_Method_PERSONAL_PROJECTS   12000 non-null  uint8 
 9   Account_Creation_Method_SIGNUP              12000 non-null  uint8 
 10  Account_Creation_Metho

### Finding the Best Features

#### SelectKbest

In [51]:
X= user.drop('Adopted_User', axis = 1)
y = user['Adopted_User']

BestFeatures = SelectKBest(score_func=chi2, k = 5)
fit_Model = BestFeatures.fit(X, y)


In [57]:
dfscores = pd.DataFrame(fit_Model.scores_)
dfcolumns = pd.DataFrame(X.columns)


In [64]:
Results = pd.concat([dfcolumns, dfscores], axis = 1)
Results.columns = ['Feature_Name', 'Feature_Score']
Results.set_index('Feature_Name', inplace = True)

In [68]:
Results.sort_values('Feature_Score', ascending = False)

,Feature_Score
Feature_Name,
Accessed_Atleast_Once,183.142851
Account_Creation_Q2,62.078674
Account_Creation_Method_PERSONAL_PROJECTS,56.694032
Account_Creation_Q3,18.656831
Account_Creation_Method_SIGNUP_GOOGLE_AUTH,13.908834
Account_Creation_Q4,12.333829
Account_Creation_Method_SIGNUP,0.788815
Mailing_List,0.703419
Marketing_Dip,0.441751
